# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [30]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [31]:
# your code here
orders = pd.read_csv("Orders.zip")
print(orders.head())
orders.shape

   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

(397924, 14)

In [32]:
orders["CustomerID"].unique()
orders["CustomerID"].nunique()

4339

In [33]:
sum_spent = orders.groupby("CustomerID", as_index=False).agg({"amount_spent":"sum"})
sum_spent

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [46]:
labels = ['Preferred Customer','VIP']

parameters = [0.75, 0.95, 1]

orders["Customer_label"] = pd.qcut(sum_spent["amount_spent"], parameters, labels=labels)

orders["Customer_label"]

0                        VIP
1         Preferred Customer
2         Preferred Customer
3         Preferred Customer
4                        NaN
                 ...        
397919                   NaN
397920                   NaN
397921                   NaN
397922                   NaN
397923                   NaN
Name: Customer_label, Length: 397924, dtype: category
Categories (2, object): [Preferred Customer < VIP]

In [47]:
orders[["Customer_label", "CustomerID"]].dropna()

,Customer_label,CustomerID
0,VIP,17850
1,Preferred Customer,17850
2,Preferred Customer,17850
3,Preferred Customer,17850
5,Preferred Customer,17850
...,...,...
4319,Preferred Customer,14083
4320,Preferred Customer,14083
4328,Preferred Customer,13013
4337,Preferred Customer,13013


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [48]:
# your code here
# po unique id customer saberi sve kupovine da bi dobila koliko su ukupno potrosili
# istrazi i onda napravi bins kao mala potrosnja srednja velika itd
# onda obelezi ove sa 95 kao vip i ove malo 75-95 kao preferred
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer_label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred Customer
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred Customer
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred Customer
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN


In [49]:
country_vip = orders[(orders["Customer_label"] == "VIP")]

country_vip_count = country_vip["Country"].value_counts()

country_vip_count.head(1)

United Kingdom    204
Name: Country, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [50]:
# your code here
# od ovih vip izvuci zemlje i broj


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [52]:
# your code here
# isto samo za oba i saberi

country_vip_pref = orders[(orders["Customer_label"] == "VIP") | (orders["Customer_label"] == "Preferred Customer")]

country_vip_pref_count = country_vip_pref["Country"].value_counts()

country_vip_pref_count.head(1)

United Kingdom    1035
Name: Country, dtype: int64